In [21]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [22]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [23]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [24]:
model_tag = 'cohere'  
dataset_tag = 'go_emotions'
prompt = 'emb'
temp = 0
number_of_experimet = 1
selected_samples =  'page_rank'

In [25]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [26]:
dataset_for_annotation.columns

Index(['text', 'label'], dtype='object')

In [27]:
print(model, dataset_for_annotation, prompt_txt, examples_for_prompt, output_path)

CohereLLM
Params: {}                                                    text        label
0     Ok.... he can *call* it the State of the Union...  disapproval
1                                 Wheres the telescope?    curiosity
2     > Why create more jobs when no one is availabl...    confusion
3        [NAME] deserves to play for a proper club ffs.   admiration
4     I'm sorry, do people NOT listen to [NAME] 9th ...    gratitude
...                                                 ...          ...
1915  Yeah, i dont know bout you guys, but my parent...     approval
1916  This is hilarious! Sounds similar to my buddy'...    amusement
1917                   But that jumper looks hella cosy          joy
1918                  Sorry. This thing knows whats up!      remorse
1919  As a household we deal with it by respecting e...     approval

[1920 rows x 2 columns] Task: Read the following text and determine the most appropriate category based on its content and the examples below.

Catego

In [28]:
text_col = "text"
label_col = "label"
output_col = "output"

In [29]:
annotator = LLMAnnotator(
    model=model,
    dataset=dataset_for_annotation,
    examples_for_prompt=examples_for_prompt,
    prompt_template=prompt_txt,
    column_text=text_col,
    column_label=label_col,
    column_output=output_col
)


In [30]:
df = annotator.get_results(is_save=True, output_path=output_path)

/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:81: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0 Predicted label: disapproval
Nr: 1 Predicted label: curiosity
Nr: 2 Predicted label: disapproval
Nr: 3 Predicted label: disapproval
Nr: 4 Predicted label: surprise
Nr: 5 Predicted label: caring
Nr: 6 Predicted label: approval
Nr: 7 Predicted label: approval
Nr: 8 Predicted label: disgust
Nr: 9 Predicted label: amusement
Nr: 10 Predicted label: amusement
Nr: 11 Predicted label: realization
Nr: 12 Predicted label: disgust
Nr: 13 Predicted label: curiosity
Nr: 14 Predicted label: approval
Nr: 15 Predicted label: disappointment
Nr: 16 Predicted label: desire
Nr: 17 Predicted label: anger
Nr: 18 Predicted label: disgust
Nr: 19 Predicted label: disappointment
Nr: 20 Predicted label: caring
Nr: 21 Predicted label: caring
Nr: 22 Predicted label: amusement
Nr: 23 Predicted label: disapproval
Nr: 24 Predicted label: disappointment
Nr: 25 Predicted label: disapproval
Nr: 26 Predicted label: disappointment
Nr: 27 Predicted label: love
Nr: 28 Predicted label: excitement
Nr: 29 Predicted label

In [31]:
annotator.save_results(output_path=output_path)

Results saved successfully to ./results/emb_temp0/cohere/go_emotions_cohere_emb_page_rank_temp0_exp1.csv


In [32]:
df = pd.read_csv(output_path)
df

,text,output,logprobs,top_logprobs,original_label
0,Ok.... he can *call* it the State of the Union...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",disapproval
1,Wheres the telescope?,curiosity,"{'content': [{'token': 'cur', 'bytes': [99, 11...","[{'token': 'cur', 'bytes': [99, 117, 114], 'lo...",curiosity
2,> Why create more jobs when no one is availabl...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",confusion
3,[NAME] deserves to play for a proper club ffs.,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",admiration
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",surprise,"{'content': [{'token': 'sur', 'bytes': [115, 1...","[{'token': 'sur', 'bytes': [115, 117, 114], 'l...",gratitude
...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",disgust,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",approval
1916,This is hilarious! Sounds similar to my buddy'...,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",amusement
1917,But that jumper looks hella cosy,desire,"{'content': [{'token': 'des', 'bytes': [100, 1...","[{'token': 'des', 'bytes': [100, 101, 115], 'l...",joy
1918,Sorry. This thing knows whats up!,remorse,"{'content': [{'token': 'rem', 'bytes': [114, 1...","[{'token': 'rem', 'bytes': [114, 101, 109], 'l...",remorse


In [33]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_first_bracketed(text):
        match = re.search(r'\[([^\[\]]+?)\]', str(text))
        return match.group(1) if match else None
    
    df[write_col] = df[read_col].apply(extract_first_bracketed)
    return df


In [34]:
import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, output_col, 'extracted_label')
    df.to_csv(path, index=False)
    
    return df


In [35]:
df_new = validate_text_column(output_path, examples_for_prompt)
df_new

,text,output,logprobs,top_logprobs,original_label,was_in_selected_samples,extracted_label
0,Ok.... he can *call* it the State of the Union...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",disapproval,False,None
1,Wheres the telescope?,curiosity,"{'content': [{'token': 'cur', 'bytes': [99, 11...","[{'token': 'cur', 'bytes': [99, 117, 114], 'lo...",curiosity,False,None
2,> Why create more jobs when no one is availabl...,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",confusion,False,None
3,[NAME] deserves to play for a proper club ffs.,disapproval,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",admiration,False,None
4,"I'm sorry, do people NOT listen to [NAME] 9th ...",surprise,"{'content': [{'token': 'sur', 'bytes': [115, 1...","[{'token': 'sur', 'bytes': [115, 117, 114], 'l...",gratitude,False,None
...,...,...,...,...,...,...,...
1915,"Yeah, i dont know bout you guys, but my parent...",disgust,"{'content': [{'token': 'dis', 'bytes': [100, 1...","[{'token': 'dis', 'bytes': [100, 105, 115], 'l...",approval,False,None
1916,This is hilarious! Sounds similar to my buddy'...,amusement,"{'content': [{'token': 'am', 'bytes': [97, 109...","[{'token': 'am', 'bytes': [97, 109], 'logprob'...",amusement,False,None
1917,But that jumper looks hella cosy,desire,"{'content': [{'token': 'des', 'bytes': [100, 1...","[{'token': 'des', 'bytes': [100, 101, 115], 'l...",joy,False,None
1918,Sorry. This thing knows whats up!,remorse,"{'content': [{'token': 'rem', 'bytes': [114, 1...","[{'token': 'rem', 'bytes': [114, 101, 109], 'l...",remorse,False,None
